In [ ]:
# This project is almost identical to the tutorial by Valerio Velardo: 

import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive') 

Mounted at /content/gdrive


In [ ]:
DATA_PATH = "/content/gdrive/MyDrive/ML_Datasets/data_10.json" # This refers to the JSON file created by using the "GTZAN_data_to_JSON" script

In [ ]:
def load_data(data_path):
    #Loads training dataset from json file.
        # data_path is a string which lists the path to json file containing data
        # X = Inputs
        # y = Targets

    with open(data_path, "r") as fp:
        data = json.load(fp)

    X = np.array(data["mfcc"])
    y = np.array(data["labels"])

    print('Shape of X: ' + X.shape)
    print('Shape of y: ' + y.shape)

    return X, y

In [ ]:
def plot_history(history):
    
    # Plots accuracy/loss for training/validation set as a function of the epochs

    fig, axs = plt.subplots(2)

    # create accuracy sublpot
    axs[0].plot(history.history["accuracy"], label="train accuracy")
    axs[0].plot(history.history["val_accuracy"], label="test accuracy")
    axs[0].set_ylabel("Accuracy")
    axs[0].legend(loc="lower right")
    axs[0].set_title("Accuracy eval")

    # create error sublpot
    axs[1].plot(history.history["loss"], label="train error")
    axs[1].plot(history.history["val_loss"], label="test error")
    axs[1].set_ylabel("Error")
    axs[1].set_xlabel("Epoch")
    axs[1].legend(loc="upper right")
    axs[1].set_title("Error eval")

    plt.show()


In [ ]:
def prepare_datasets(test_size, validation_size, X, y):
    #Loads data and splits it into train, validation and test sets.

    # test_size = Value in [0, 1] indicating percentage of data set to allocate to test split
    # validation_size = Value in [0, 1] indicating percentage of train set to allocate to validation split

    # X_train = Input training set
    # X_validation = Input validation set
    # X_test = Input test set
    # y_train = Target training set
    # y_validation = Target validation set
    # y_test = Target test set 

    # create train, validation and test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    return X_train, X_validation, X_test, y_train, y_validation, y_test


In [ ]:
def build_RNN_model(input_shape):
    # Generates the RNN-LSTM model specific to this program (fixed number of layers)
    # input_shape = Shape of input set

    # build network topology
    model = keras.Sequential()

    # 2 LSTM layers
    model.add(keras.layers.LSTM(64, input_shape=input_shape, return_sequences=True))
    # The former RNN layer or layers should set return_sequences to True so that the following RNN layer or layers can have the full sequence as input.
    model.add(keras.layers.LSTM(64))

    # dense layer
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3))

    # output layer (10 different genres in the dataset)
    model.add(keras.layers.Dense(10, activation='softmax'))

    return model

In [ ]:
if __name__ == "__main__":

    # load data
    X, y = load_data(DATA_PATH)

    print(X.shape)
    print()
    print(y.shape)
    

(9996, 130, 13)

(9996,)


In [ ]:
# get train, validation, test splits
X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25, 0.2, X, y) 
    
#25% of data used for training, 20% for validation

# create network
input_shape = (X_train.shape[1], X_train.shape[2]) # In this case, the input shape expects a 2-dimensional shape in the RNN
    
# First dimension: The dataset contains 130 samples which are audio signals over a certain time interval (30 secs)
# Second dimension: The actual coefficients that we extracted in previous processing [MFCCs (Mel Frequency Cepstral Coefficients)]

model = build_RNN_model(input_shape)

# compile model
optimiser = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimiser,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


In [ ]:
# train model
history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=32, epochs=30)

In [ ]:
# plot accuracy/error for training and validation
plot_history(history)

# evaluate model on test set
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)